# Gradient-Boosting Machine (XGBoost) - With Tuned Hyperparameters
### *Exploring the association between neoantigen-related variables and immune scores*
This notebook is the continuation of the `xgboost.ipynb` notebook, which details hyperparameter tuning grid search with cross-validation to model interactions of select X features on a subset of Y labels (based on the PCA work done by Caitlin from GB team). 

#### **Package and Raw Data Loading**
First, import necessary packages and load in the raw data table into `pandas` dataFrame. 



In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from feature_engine.transformation import YeoJohnsonTransformer

from warnings import simplefilter, filterwarnings
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
filterwarnings("ignore", category=UserWarning)
pd.set_option('display.max_columns', None)
%config InlineBackend.figure_format = 'retina'

Load up the cleaned-up dataset wrangled from MH's latest work.

In [ ]:
# read in latest data
# use the 202409_new_excludedIHC_batch-duplicate-removed.tsv
df = pd.read_csv("../input-data/SA/202409_new_excludedIHC_batch-duplicate-removed.tsv",sep="\t")
print(df.shape)

#print row 107 and 226
df.iloc[[107,226,555,872], :25]

In [ ]:
# exclude the 29 Cibersort scores, leaving only 3
df = df.drop(columns=['Bindea_full', 'Expanded_IFNg', 
        'C_Bcellsmemory','C_Plasmacells','C_TcellsCD8','C_TcellsCD4naive',
         'C_TcellsCD4memoryactivated','C_Tcellsfollicularhelper',
         'C_Tcellsregulatory(Tregs)','C_Tcellsgammadelta','C_NKcellsresting',
         'C_NKcellsactivated', 'C_Monocytes', 'C_MacrophagesM0',
         'C_MacrophagesM1','C_Dendriticcellsresting',
         'C_Dendriticcellsactivated', 'C_Mastcellsresting',
         'C_Mastcellsactivated','C_Eosinophils', 'C_Neutrophils', 'S_PAM100HRD'])

print(df.shape)
df.head()

#### **Data Preprocessing**

Decide all the clinical variables and neoantigen-related variables to keep in the X matrix (features).

1. `Subtype` column has already been encoded categorically by `HR_status` and `HER_status` columns so these two columns can be dropped. ***UPDATE: due to their lesser importance during the default XGBoost modeling, `PAM50` column was dropped as well.***

2.  `AgeGroup` is just a binned information of `Age` column so it is dropped as it is redundant.

3. Drop `FusionNeo_bestScore`, `FusionTransscript_Count`, `Fusion_T2NeoRate` columns as well as the `SNVindelNeo_IC50` and `SNVindelNeo_IC50Percentile` columns for now to reduce complexity.

4. Drop `Batch` column.

> **UPDATE 1: Exclude `TotalNeo_Count`, and include `Fusion_T2NeoRate` and `SNVindelNeo_IC50` columns. Also, rename `Fusion_T2NeoRate` to `FN/FT_Ratio`.**

> **UPDATE 2: put back `FusionNeo_bestScore` into the X variable set and rename it into `FusionNeo_bestIC50`**

In [ ]:
# let's drop all NaN for now and set col 'ID' as index
dfd = df.drop(columns = ['Batch', 'Stage', 'PAM50', 'HR_status', 'HER_status', 'AgeGroup', 'TotalNeo_Count', 'FusionTransscript_Count', 'SNVindelNeo_IC50Percentile']).dropna().set_index('ID')
print(dfd.shape)
dfd.head()

In [5]:
# rename the column `Fusion_T2NeoRate` to `FN/FT_Ratio` and `FusionNeo_bestScore` to `FusionNeo_bestIC50`
dfd.rename(columns={'Fusion_T2NeoRate': 'FN/FT_Ratio'}, inplace=True)
dfd.rename(columns={'FusionNeo_bestScore': 'FusionNeo_bestIC50'}, inplace=True)

In [ ]:
dfd.head()

**Sanity Check:** Check to make sure there is no duplicated index rows in the dataset.

In [ ]:
print(dfd.index[dfd.index.duplicated()].unique())
rows_dupe = list(dfd.index[dfd.index.duplicated()].unique())
rows_dupe

Now we can encode categorical variables and integer variables accordingly so they are more amenable to machine learning algorithms. 

Check data types of the resulting cleaned up dataFrame.

In [ ]:
# check original df's dtypes
pd.set_option('display.max_rows', None)
dfd.dtypes

We need to encode the `object` columns of `Subtype` and `FN/FT_Ratio` into appropriate types. Change `Age`, `TumorGrade`, and `IMPRES` into `int64` as well as all `*_Count` columns because they are discrete variables. Change the `FN/FT_Ratio` into `float64`.

In [ ]:
dfd['Subtype'] = dfd['Subtype'].astype('category')
dfd['Age'] = dfd['Age'].astype('int64')
dfd['TumorGrade'] = dfd['TumorGrade'].astype('int64')
dfd['IMPRES'] = dfd['IMPRES'].astype('int64')
dfd['FusionNeo_Count'] = dfd['FusionNeo_Count'].astype('int64')
dfd['SNVindelNeo_Count'] = dfd['SNVindelNeo_Count'].astype('int64')
dfd['FN/FT_Ratio'] = dfd['FN/FT_Ratio'].astype('float64')

print(dfd.dtypes)
pd.set_option('display.max_rows', 8)

Now we can use Feature_Engine's `OneHotEncoder()` to create a `k` dummy variable set for `Subtype`.

**NOTE**: The encoded columns will be appended at the end of the dataFrame. 


In [ ]:
from feature_engine.encoding import OneHotEncoder

encoder = OneHotEncoder(
    variables=['Subtype'],
    drop_last=False)

encoder.fit(dfd)
dfd_ = encoder.transform(dfd)
dfd_.head()

In [11]:
# Specify the encoded columns to shift
enc_cols = ['Subtype_HR+/HER2-', 'Subtype_HR+/HER2+', 'Subtype_TNBC', 'Subtype_HR-/HER2+']

# Drop the specified columns and store them
encoded_df = dfd_[enc_cols]
dfenc = dfd.drop(columns=['Subtype'])

# Specify the index where you want to reinsert the columns
insert_index = 0  # This will insert at the first column

# Reinsert the columns
for i, col in enumerate(encoded_df.columns):
    dfenc.insert(insert_index + i, col, encoded_df[col])

Below is the categorically-encoded dataframe.

In [ ]:
print(dfenc.shape)
dfenc.head()

And below is the original, unencoded dataframe.

In [ ]:
print(dfd.shape)
dfd.head()

#### **Subsetting Y Labels for XGBoost**

In the previous exploration, many of the immune scores (Y targets/labels) might not really show much relationship with fusion neoantigen variables so they may not be as informative. We decided to use Caitlin's finding and subset the Y labels into several clinically meaningful groups.

In [ ]:
# use the unencoded categorical dataframe (dfd) and drop the Subtype categorical column
df_dcat = dfd.drop(columns=['Subtype'])
print(df_dcat.shape)
df_dcat.head()

First list all the clinical variables that would be the X feature set.

In [15]:
X_features = ['Subtype_HR+/HER2-', 'Subtype_HR+/HER2+', 'Subtype_TNBC', 'Subtype_HR-/HER2+', 'Age', 'TumorGrade', 'TumourSize', 'FusionNeo_Count', 'FusionNeo_bestIC50', 'FN/FT_Ratio', 'SNVindelNeo_Count', 'SNVindelNeo_IC50']

In [16]:
X_features_nocat = ['Age', 'TumorGrade', 'TumourSize', 'FusionNeo_Count', 'FusionNeo_bestIC50', 'FN/FT_Ratio', 'SNVindelNeo_Count', 'SNVindelNeo_IC50']

In [ ]:
# Now get the Y variable set
Y_labels_all = [col for col in dfd.drop(columns=['Subtype']).columns if col not in X_features]
print(Y_labels_all[:5])
len(Y_labels_all)

In [ ]:
# load up the tsv containing the groupings of the different immune scores
df_imscores = pd.read_csv('../input-data/SA/immune_score_groupings.tsv', sep='\t')
df_imscores.head()

In [ ]:
df_imscores.columns

In [20]:
# now convert each column into a Series and drop NA
# Create a dictionary to store the Series
imscore_series_dict = {}

# Iterate through each column in the DataFrame
for column in df_imscores.columns:
    # Convert the column to a Series, drop NaN values, and store in the dictionary
    imscore_series_dict[column] = df_imscores[column].dropna().tolist()


#### **Spearman Correlation Heatmap**

Before moving on with XGBoost, we can plot a Spearman correlation matrix between the clinical variables (X features) and the different groupings of immune score variables (Y labels).

First, draw a matrix on the full cleaned up dataframe.

In [ ]:
# replot heatmap on transformed data
corr_df = df_dcat.corr(method='spearman')
corr_df = corr_df.round(2)

# Create a mask for the upper triangle
mask = np.triu(np.ones_like(corr_df, dtype=bool))

plt.figure(figsize=(46, 36))

# Create the correlation matrix and represent it as a heatmap.
hm = sns.heatmap(corr_df, annot = False, cmap = 'RdBu_r', square = True, linewidths=0.5, center=0, mask=mask, cbar_kws={"shrink": .5})

# Get current labels
ylabels = hm.get_yticklabels()
xlabels = hm.get_xticklabels()

# Hide the first y-axis label and the last x-axis label
ylabels[0].set_visible(False)
xlabels[-1].set_visible(False)

# Rotate and align the tick labels
plt.setp(xlabels, rotation=45, ha='right')

# Define columns to highlight and their colors
highlight_cols = {
    "FN/FT_Ratio": "crimson",
    "FusionNeo_Count": "olive",
    "SNVindelNeo_Count": "darkviolet"
}

# Change color of specific x-axis and y-axis labels
for label in xlabels:
    if label.get_text() in highlight_cols:
        label.set_color(highlight_cols[label.get_text()])
        label.set_fontweight('bold')

for label in ylabels:
    if label.get_text() in highlight_cols:
        label.set_color(highlight_cols[label.get_text()])
        label.set_fontweight('bold')

# Removes all ticks
hm.tick_params(left=False, bottom=False)

hm.set_title('Dataframe Correlation Heatmap', fontsize=30, x=0.45)

plt.show()

Now, draw the same on the subset Y labels grouped as `activator_T`.

In [ ]:
# get the list from the dict
activator_t = imscore_series_dict['activator_T']
suppressor_t = imscore_series_dict['suppressor_T']
best_prog = imscore_series_dict['HR<1_best_10_prog']
worst_prog = imscore_series_dict['HR>1_worst_10_prog']

# combine these with the X feature set (unique only using set())
merged_cols = X_features_nocat + activator_t + suppressor_t + best_prog + worst_prog
merged_cols = list(set(merged_cols))
print(f"Total number of elements in merged_cols (unsorted): {len(merged_cols)}")

# there are repeated immune scores (at least in between two groups, can be more than two groups) so get a list of them first
from itertools import combinations
# list of all the sets
all_sets = [set(activator_t), set(suppressor_t), set(best_prog), set(worst_prog)]

# Get all possible combinations of 2 sets
set_combo = combinations(all_sets, 2)

# Find the union of all set combinations
union_of_combo = list(set.union(*[set.intersection(c1, c2) for c1, c2 in set_combo]))

print(f"Elements that overlap between at least two sets: {union_of_combo}")

# rearrange the list element order based on another list
merged_cols = [x for x in X_features_nocat] + union_of_combo + [x for x in activator_t if x not in union_of_combo] + [x for x in suppressor_t if x not in union_of_combo] + [x for x in best_prog if x not in union_of_combo] + [x for x in worst_prog if x not in union_of_combo]

print(f"Total number of elements in merged_cols (sorted by original X feature order and groups): {len(merged_cols)}")


In [ ]:
# now subset the dcat data
df_dcat_ss = df_dcat[merged_cols]
df_dcat_ss.head()

In [ ]:
# replot heatmap on transformed data
corr_df = df_dcat_ss.corr(method='spearman')
corr_df = corr_df.round(2)

# Create a mask for the upper triangle
mask = np.triu(np.ones_like(corr_df, dtype=bool))

plt.figure(figsize=(46, 36))

# Create the correlation matrix and represent it as a heatmap.
hm = sns.heatmap(corr_df, annot = False, cmap = 'RdBu_r', square = True, linewidths=0.5, center=0, mask=mask, cbar_kws={"shrink": .5})

# Get current labels
ylabels = hm.get_yticklabels()
xlabels = hm.get_xticklabels()

# Hide the first y-axis label and the last x-axis label
ylabels[0].set_visible(False)
xlabels[-1].set_visible(False)

# Rotate and align the tick labels
plt.setp(xlabels, rotation=45, ha='right')

# Define columns to highlight and their colors
highlight_cols = {
    "FN/FT_Ratio": "crimson",
    "FusionNeo_Count": "olive",
    "SNVindelNeo_Count": "darkviolet"
}

# Change color of specific x-axis and y-axis labels
for label in xlabels:
    if label.get_text() in highlight_cols:
        label.set_color(highlight_cols[label.get_text()])
        label.set_fontweight('bold')

for label in ylabels:
    if label.get_text() in highlight_cols:
        label.set_color(highlight_cols[label.get_text()])
        label.set_fontweight('bold')

# Removes all ticks
hm.tick_params(left=False, bottom=False)

hm.set_title('Dataframe Correlation Heatmap', fontsize=30, x=0.45)

plt.show()

#### **Split Dataset with `train_test_split`**

Split the dataset before modeling to avoid information leakage, then preprocess the data through the set up Pipeline before XGBoost.

In [ ]:
# subset X features; use the list generated before
X = dfenc[X_features]
X

Now grab the Y targets (do this as a whole, but we will train on each column individually later).

In [ ]:
# Now get the Y variable set
Y = dfenc[Y_labels_all]
Y

Now we perform train test split on the X and Y variables.

In [27]:
# Perform train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
Y_train.head()

In [ ]:
Y_test.head()

As we don't want to transform all the X columns (because some of them are discrete numerical data and some of them are one-hot encoded categorical variables), we need to specify the columns to transform.

In [ ]:
X.info()

#### **Create a Data Transformation Pipeline from `feature_engine` Package**
First, the pipeline will apply the Yeo-Johnson transformation on the split datasets on select X features and all Y labels, and scale them using `StandardScaler` (but wrapped within `feature_engine`'s wrapper) on select X features and all Y labels.

This pipeline would enable easy inverse transform steps for both X and Y datasets later.

In [33]:
X_vars_to_transform = ['TumourSize', 'FusionNeo_Count', 'FusionNeo_bestIC50', 'FN/FT_Ratio', 'SNVindelNeo_Count', 'SNVindelNeo_IC50']

Do for X datasets.

In [34]:
from feature_engine.pipeline import Pipeline
from feature_engine.transformation import YeoJohnsonTransformer
from feature_engine.wrappers import SklearnTransformerWrapper
from sklearn.preprocessing import StandardScaler

# select variables to scale
scale_cols_X = X_train.columns.tolist()
scale_cols_X = [col for col in scale_cols_X if col not in ['Age', 'TumorGrade', 'Subtype_HR+/HER2-', 'Subtype_HR+/HER2+', 'Subtype_TNBC', 'Subtype_HR-/HER2+']]

# Create the pipeline
preprocess_pipeline_X = Pipeline([
    ('yeo_johnson', YeoJohnsonTransformer(variables=X_vars_to_transform)),
    ('scaler', SklearnTransformerWrapper(transformer = StandardScaler(), variables = scale_cols_X))
])

# Fit the pipeline to the training data
preprocess_pipeline_X.fit(X_train)

# Transform the training data
X_train_yjs = preprocess_pipeline_X.transform(X_train)
# Transform the test data
X_test_yjs = preprocess_pipeline_X.transform(X_test)


In [ ]:
X_train

In [ ]:
X_train_yjs

In [37]:
# select variables to scale
scale_cols_Y = Y_train.columns.tolist()

# Create the pipeline
preprocess_pipeline_Y = Pipeline([
    ('yeo_johnson', YeoJohnsonTransformer()),
    ('scaler', SklearnTransformerWrapper(transformer = StandardScaler(), variables = scale_cols_Y))
])

# Fit the pipeline to the training data
preprocess_pipeline_Y.fit(Y_train)

# Transform the training data
Y_train_yjs = preprocess_pipeline_Y.transform(Y_train)
# Transform the test data
Y_test_yjs = preprocess_pipeline_Y.transform(Y_test)

In [ ]:
Y_train

In [ ]:
Y_train_yjs

#### **XGBoost Learning**

Time to test XGBoost. Select `S_Buck14_score` column as the first target/label (`y`) variable first.

In [ ]:
from xgboost import XGBRegressor
from xgboost import plot_importance

y_target = 'S_Buck14_score'
y_train_tg = Y_train_yjs[y_target]
y_test_tg = Y_test_yjs[y_target]

model_xgb = XGBRegressor(n_estimators=150, random_state=42)

# fit
model_xgb.fit(X_train_yjs, y_train_tg)

# predict
y_train_transformed_pred = model_xgb.predict(X_train_yjs)
y_test_transformed_pred = model_xgb.predict(X_test_yjs)

# Create a DataFrame with the same columns as the original y used in fit
y_train_dummy_df = pd.DataFrame(0, index=X_train_yjs.index, columns=Y_train_yjs.columns)
y_train_dummy_df[y_target] = y_train_transformed_pred

y_test_dummy_df = pd.DataFrame(0, index=X_test_yjs.index, columns=Y_test_yjs.columns)
y_test_dummy_df[y_target] = y_test_transformed_pred

# apply inverse transform
y_train_dummy_df_inv = preprocess_pipeline_Y.inverse_transform(y_train_dummy_df)
y_test_dummy_df_inv = preprocess_pipeline_Y.inverse_transform(y_test_dummy_df)

# Extract the relevant target column
y_train_pred = y_train_dummy_df_inv[y_target].to_numpy()
y_test_pred = y_test_dummy_df_inv[y_target].to_numpy()

# Evaluate
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Calculate metrics
train_r2 = r2_score(Y_train[y_target], y_train_pred)
test_r2 = r2_score(Y_test[y_target], y_test_pred)

train_rmse = np.sqrt(mean_squared_error(Y_train[y_target], y_train_pred))
test_rmse = np.sqrt(mean_squared_error(Y_test[y_target], y_test_pred))

train_mae = mean_absolute_error(Y_train[y_target], y_train_pred)
test_mae = mean_absolute_error(Y_test[y_target], y_test_pred)

# Print results
print("Model Performance:")
print(f"{'Metric':<10} {'Train':<10} {'Test':<10}")
print("-" * 30)
print(f"{'R2':<10} {train_r2:<10.4f} {test_r2:<10.4f}")
print(f"{'RMSE':<10} {train_rmse:<10.4f} {test_rmse:<10.4f}")
print(f"{'MAE':<10} {train_mae:<10.4f} {test_mae:<10.4f}")

# Plot actual vs predicted
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.scatter(Y_train[y_target], y_train_pred, alpha=0.5)
ax1.plot([Y_train[y_target].min(), Y_train[y_target].max()], [Y_train[y_target].min(), Y_train[y_target].max()], 'r--', lw=2)
ax1.set_xlabel('Actual')
ax1.set_ylabel('Predicted')
ax1.set_title('Train Set')

ax2.scatter(Y_test[y_target], y_test_pred, alpha=0.5)
ax2.plot([Y_test[y_target].min(), Y_test[y_target].max()], [Y_test[y_target].min(), Y_test[y_target].max()], 'r--', lw=2)
ax2.set_xlabel('Actual')
ax2.set_ylabel('Predicted')
ax2.set_title('Test Set')

plt.tight_layout()
plt.show()

# Feature importance
fig, ax = plt.subplots(figsize=(18, 16), dpi=300)
plot_importance(model_xgb, ax=ax)
plt.show()

import shap
# Create the SHAP explainer
explainer = shap.TreeExplainer(model_xgb)

# run explanation object on X_test dataset because we don't want to learn what the model learned from the X_train data, but to see features that would influence predictions on new data
shap_values = explainer(X_test_yjs)

plt.figure(figsize=(18, 16)) 
# Summary plot
shap.summary_plot(shap_values, show=False)
plt.tight_layout()
plt.show()

# Perform clustering
clust = shap.utils.hclust(X_test_yjs, y_test_tg)
# Create the bar plot with clustering
plt.figure(figsize=(18, 16)) 
shap.plots.bar(shap_values, clustering=clust, clustering_cutoff=1, show=False)
plt.tight_layout()
plt.show()

#### **Iterative Learning over all Y labels**

The learning using XGBoost above was done on just one Y label, which is the `ESTIMATE` column. Let's put these into a set of functions so we can run this process iteratively on all Y columns we have set up.

In [41]:
# copy the steps above as a function
import os 
import shap
from xgboost import XGBRegressor
from sklearn.model_selection import learning_curve
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

class YTargetMetrics:
    def __init__(self, target_name, train_r2, test_r2, train_rmse, test_rmse, train_mae, test_mae):
        self.target_name = target_name
        self.train_r2 = train_r2
        self.test_r2 = test_r2
        self.train_rmse = train_rmse
        self.test_rmse = test_rmse
        self.train_mae = train_mae
        self.test_mae = test_mae

    def __str__(self):
        return f"""Model Performance for {self.target_name}:
{'Metric':<10} {'Train':<10} {'Test':<10}
{'-' * 30}
{'R2':<10} {self.train_r2:<10.4f} {self.test_r2:<10.4f}
{'RMSE':<10} {self.train_rmse:<10.4f} {self.test_rmse:<10.4f}
{'MAE':<10} {self.train_mae:<10.4f} {self.test_mae:<10.4f}"""

    def to_dict(self):
        return {
            'target_name': self.target_name,
            'train_r2': self.train_r2,
            'test_r2': self.test_r2,
            'train_rmse': self.train_rmse,
            'test_rmse': self.test_rmse,
            'train_mae': self.train_mae,
            'test_mae': self.test_mae
        }

def plot_learning_curves(estimator, X, y, target, cv=5, train_sizes=np.linspace(.1, 1.0, 5)):
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=-1, train_sizes=train_sizes,
        scoring='neg_mean_squared_error')
    
    train_scores_mean = -train_scores.mean(axis=1)
    train_scores_std = train_scores.std(axis=1)
    test_scores_mean = -test_scores.mean(axis=1)
    test_scores_std = test_scores.std(axis=1)

    plt.figure(figsize=(10, 6), dpi=300)
    plt.title("Learning Curves")
    plt.xlabel("Training examples")
    plt.ylabel("Mean Squared Error")
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1, color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")
    plt.legend(loc="best")

    # Create 'plots' directory if it doesn't exist
    os.makedirs(f'plots/{target}', exist_ok=True)

    # Save the plot
    plt.savefig(f'plots/{target}/{target}-xgb-def-model-learning-curve.png')
    plt.close()

def run_xgboost_model(y_target, Y_train, Y_test, X_train_transformed, X_test_transformed, Y_train_transformed, Y_test_transformed, preprocess_pipeline_Y):
    # assign untransformed, raw target data
    raw_y_train = Y_train[y_target]
    raw_y_test = Y_test[y_target]

    model_instance = XGBRegressor(n_estimators=150, random_state=42)

    # fit
    model_instance.fit(X_train_transformed, Y_train_transformed[y_target])
    
    # predict
    y_train_pred_transformed = model_instance.predict(X_train_transformed)
    y_test_pred_transformed = model_instance.predict(X_test_transformed)

    # Create a DataFrame with the same columns as the original y used in fit
    dummy_train_y = pd.DataFrame(0, index=X_train_transformed.index, columns=Y_train_transformed.columns)
    dummy_train_y[y_target] = y_train_pred_transformed

    dummy_test_y = pd.DataFrame(0, index=X_test_transformed.index, columns=Y_test_transformed.columns)
    dummy_test_y[y_target] = y_test_pred_transformed

    # apply inverse transform
    dummy_train_y_inv = preprocess_pipeline_Y.inverse_transform(dummy_train_y)
    dummy_test_y_inv = preprocess_pipeline_Y.inverse_transform(dummy_test_y)

    # Extract the relevant target column
    y_train_pred = dummy_train_y_inv[y_target].to_numpy()
    y_test_pred = dummy_test_y_inv[y_target].to_numpy()

    # Evaluate model
    # Calculate metrics
    train_r2 = r2_score(raw_y_train, y_train_pred)
    test_r2 = r2_score(raw_y_test, y_test_pred)

    train_rmse = np.sqrt(mean_squared_error(raw_y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(raw_y_test, y_test_pred))

    train_mae = mean_absolute_error(raw_y_train, y_train_pred)
    test_mae = mean_absolute_error(raw_y_test, y_test_pred)

    # Plot learning curves
    plot_learning_curves(model_instance, X_train_transformed, Y_train_transformed[y_target], y_target)

    # Print results
    # print("Model Performance:")
    # print(f"{'Metric':<10} {'Train':<10} {'Test':<10}")
    # print("-" * 30)
    # print(f"{'R2':<10} {train_r2:<10.4f} {test_r2:<10.4f}")
    # print(f"{'RMSE':<10} {train_rmse:<10.4f} {test_rmse:<10.4f}")
    # print(f"{'MAE':<10} {train_mae:<10.4f} {test_mae:<10.4f}")

    # Plot actual vs predicted
    _, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), dpi=300)

    ax1.scatter(raw_y_train, y_train_pred, alpha=0.5)
    ax1.plot([raw_y_train.min(), raw_y_train.max()], [raw_y_train.min(), raw_y_train.max()], 'r--', lw=2)
    ax1.set_xlabel('Actual')
    ax1.set_ylabel('Predicted')
    ax1.set_title('Training Set')

    ax2.scatter(raw_y_test, y_test_pred, alpha=0.5)
    ax2.plot([raw_y_test.min(), raw_y_test.max()], [raw_y_test.min(), raw_y_test.max()], 'r--', lw=2)
    ax2.set_xlabel('Actual')
    ax2.set_ylabel('Predicted')
    ax2.set_title('Testing Set')

    plt.tight_layout()

    # Create 'plots' directory if it doesn't exist
    os.makedirs(f'plots/{y_target}', exist_ok=True)

    plt.savefig(f'plots/{y_target}/{y_target}-xgb-def-model-performance-comparison.png')
    plt.close()

    # Feature importance
    _, ax = plt.subplots(figsize=(18, 16), dpi=300)
    plot_importance(model_instance, ax=ax)

    plt.savefig(f"plots/{y_target}/{y_target}-xgb-def-model-test-set-feature-importance.png")
    plt.close()

    # Create the SHAP explainer
    explainer = shap.TreeExplainer(model_instance)

    # run explanation object on X_test dataset because we don't want to learn what the model learned from the X_train data, but to see features that would influence predictions on new data
    shap_values = explainer(X_test_transformed)

    plt.figure(figsize=(18, 16), dpi=300) 
    # Summary plot
    shap.summary_plot(shap_values, show=False)
    plt.tight_layout()
    plt.savefig(f"plots/{y_target}/{y_target}-xgb-def-model-test-set-SHAP-beeswarm.png", bbox_inches='tight')
    plt.close()

    # Perform clustering
    clust = shap.utils.hclust(X_test_transformed, Y_test_transformed[y_target])
    # Create the bar plot with clustering
    plt.figure(figsize=(18, 16)) 
    shap.plots.bar(shap_values, clustering=clust, clustering_cutoff=1, show=False)
    plt.tight_layout()
    plt.savefig(f"plots/{y_target}/{y_target}-xgb-def-model-test-set-SHAP-summary.png", dpi=300, bbox_inches='tight')
    plt.close()

    print(f"Model training and evaluation for {y_target} completed.")

    # Instead of returning a tuple, return a YTargetMetrics object
    return YTargetMetrics(y_target, train_r2, test_r2, train_rmse, test_rmse, train_mae, test_mae)


#### **Grid Search: Using `GridSearchCV` for Hyperparameter Tuning**

In [47]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

class YTargetMetrics:
    def __init__(self, target_name, test_mae, test_rmse, test_r2):
        self.target_name = target_name
        self.test_mae = test_mae
        self.test_rmse = test_rmse
        self.test_r2 = test_r2

    def __str__(self):
        return f"""Model Performance for {self.target_name}:
{'Metric':<10} {'Test':<10}
{'-' * 20}
{'MAE':<10} {self.test_mae:<10.4f}
{'RMSE':<10} {self.test_rmse:<10.4f}
{'R2':<10} {self.test_r2:<10.4f}"""

    def to_dict(self):
        return {
            'target_name': self.target_name,
            'test_mae': self.test_mae,
            'test_rmse': self.test_rmse,
            'test_r2': self.test_r2
        }


def run_grid_search_cv(model_instance, target, search_space, X_train_transformed, Y_train_transformed, output_path):
    # assign y target column
    y_train_target = Y_train_transformed[target]
    
    # Set up grid search
    grid_search = GridSearchCV(
    estimator=model_instance,
    param_grid=search_space,
    cv=5,
    refit='r2',
    scoring=['r2', 'neg_root_mean_squared_error'],
    n_jobs=-1,  # Use all available cores
    verbose=1
    )
    
    print(f"Running grid search on {target} column...")
    # Fit grid search
    grid_search.fit(X_train_transformed, y_train_target)

    # Get best parameters and model
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_
    best_metric = grid_search.best_score_

    print("Best parameters:", best_params)
    print("Best score:", best_metric)

    # get CV results into a csv file
    cv_results = pd.DataFrame(grid_search.cv_results_)
    cv_results = cv_results.sort_values(by='rank_test_r2')
    cv_results.to_csv(f'{output_path}/grid_search_results_{target}_score-{best_metric}.csv', index=False)
    return best_model, best_params, best_metric

def predict_with_best_xgboost_model(best_model, target, X_test_transformed, Y_test_transformed, Y_test, preprocess_pipeline_Y):
    # Make predictions on the test set using the best model
    y_pred_transformed = best_model.predict(X_test_transformed)

    # Create a DataFrame with the same columns as the original y used in preprocess_pipeline to reverse transformation
    dummy_df = pd.DataFrame(0, index=X_test_transformed.index, columns=Y_test_transformed.columns)
    dummy_df[target] = y_pred_transformed

    # apply inverse transform
    dummy_df_inv = preprocess_pipeline_Y.inverse_transform(dummy_df)

    # Extract the relevant target column
    y_pred = dummy_df_inv[target].to_numpy()

    # Evaluate
    y_test = Y_test[target]
    test_mae = mean_absolute_error(y_test, y_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    test_r2 = r2_score(y_test, y_pred)

    print(f"Mean Absolute Error (MAE) [Testing]: {test_mae:.2f}")
    print(f"Root Mean Squared Error (RMSE) [Testing]: {test_rmse:.2f}")
    print(f"Coefficient of Determination (R2) [Testing]: {test_r2:.2f}")

    # Instead of returning a tuple, return a YTargetMetrics object
    return YTargetMetrics(target, test_mae, test_rmse, test_r2)

In [ ]:
# # Instantiate base model
# model_xgb = XGBRegressor(random_state=42)

# # Define search space
# search_space = {
#     'n_estimators': [100, 200, 400],
#     'max_depth': [3, 6, 9],
#     'learning_rate': [0.01, 0.1, 0.5, 1],
#     'subsample': [0.5, 0.8, 1.0],
#     'gamma': [0.01, 0.1]
# }
# # set output folder
# output_loc = '../nb/out_files'
# # initialize an empty dict
# tuned_results = {}

# for target in Y_labels_all:
#     best_model, best_params, best_metric = run_grid_search_cv(model_xgb, target, search_space, X_train_yjs, Y_train_yjs, output_loc)
    
#     tuned_metrics = predict_with_best_xgboost_model(best_model, target, X_test_yjs, Y_test_yjs, Y_test, preprocess_pipeline_Y)

#     # Store the metric result in the dictionary
#     tuned_results[target] = tuned_metrics

In [ ]:
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Assuming tuned_results is your dictionary of YTargetMetrics instances

# # Extract R2 scores into a DataFrame
# r2_data = {target: metrics.test_r2 for target, metrics in tuned_results.items()}
# df_r2 = pd.DataFrame(list(r2_data.items()), columns=['Target', 'R2'])

# # Create a heatmap
# plt.figure(figsize=(12, 8))
# heatmap_data = df_r2.set_index('Target')
# sns.heatmap(heatmap_data, annot=True, cmap='YlGnBu', fmt='.4f')
# plt.title('R2 Scores Heatmap for Different Targets')
# plt.show()

# # Create a bar plot
# plt.figure(figsize=(12, 8))
# sns.barplot(x='Target', y='R2', data=df_r2)
# plt.title('R2 Scores Comparison for Different Targets')
# plt.xticks(rotation=45, ha='right')
# plt.tight_layout()
# plt.show()